In [1]:
%load_ext sql

# Connecting to Redshift CLuster

In [ ]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))
(HOST,DB_NAME, DB_USER, DB_PASSWORD, DB_PORT) = config['CLUSTER'].values()

conn_string="postgresql://{}:{}@{}:{}/{}".format(DB_USER, DB_PASSWORD, HOST, DB_PORT,DB_NAME)
print(conn_string)
%sql $conn_string

In [3]:
%run create_tables.py

'ST_Events' table created successfully...!!!
'ST_Songs' table created successfully...!!!
'users' table created successfully...!!!
'songs' table created successfully...!!!
'artists' table created successfully...!!!
'time' table created successfully...!!!
'songplays' table created successfully...!!!


In [4]:
%run etl.py

'ST_Events' COPY Successful...!!!
'ST_Songs' COPY Successful...!!!
'songplays'  Insert Successful...!!!
'users'  Insert Successful...!!!
'songs'  Insert Successful...!!!
'artists'  Insert Successful...!!!
'time'  Insert Successful...!!!
Duplicate records for userIds : ('29', '36', '15', '88', '80', '16', '49', '85')
... Deleting duplicates .....
DELETE FROM users WHERE user_id IN ('29', '36', '15', '88', '80', '16', '49', '85')
Inserted with the latest information for the updated users
*************** ETL process completed ***************


# Check if all the tables are loaded

In [ ]:
%%sql 
SELECT 'ST_EVENTS' as TABLE, COUNT(*) total_records from ST_Events 
UNION ALL
SELECT 'ST_Songs', COUNT(*) from ST_Songs;

In [ ]:
%%sql SELECT 'songplays' as table, COUNT(*) total_records from songplays
    UNION ALL
    SELECT 'users', COUNT(*) from users
    UNION ALL
    SELECT 'songs', COUNT(*)from songs
    UNION ALL
    SELECT 'artists', COUNT(*)from artists
    UNION ALL
    SELECT 'time', COUNT(*) from time

### Query to check if there are any duplicate records for the same user

In [ ]:
%sql select * from (SELECT user_id,count(*) c FROM users group by user_id ) where c>1;

## Connecting with python

In [4]:
import psycopg2

conn = psycopg2.connect("host={} dbname={} user={} password={} port={}".format(*config['CLUSTER'].values()))
cur = conn.cursor()

# Number of rows in events staging table
cur.execute("select count(*) from ST_Events")
cur.fetchall()

[(8056,)]